# Advanced techniques to improve RAG Performance

# Basic RAG

## Preparation

Documentation for RAG with LangChain: [RAG quickstart](https://python.langchain.com/docs/use_cases/question_answering/quickstart)

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Recommended: create new virtualenv
* pyenv virtualenv 3.11.4 your_venv_name
* pyenv activate your_venv_name
* pip install jupyterlab
* jupyter lab

#### Install Required Modules

In [55]:
#!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

#### Recommended: activate LangSmith from the beginning
* LangSmith credential in the .env file:
    * LANGCHAIN_TRACING_V2=true
    * LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
    * LANGCHAIN_API_KEY=your_key
    * LANGCHAIN_PROJECT=your_project_name

In [56]:
#!pip install langsmith

#### Load the necessary modules

In [4]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

## Phase 1: Indexing

#### Load the private document. In this case, we will use a web page

In [5]:
# Load document, in this case from web page
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

#### Split the document in small chunks of text

In [6]:
# Split document in small chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

#### Convert the chunks of text in numbers (embeddings) and load them into a vector database

In [7]:
# Convert small chunks in numbers (embeddings) and store in vector database
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

## Phase 2: Retrieval

#### Set the retriever
A "retriever" refers to a component or object that is used to retrieve (or search for) specific information from a database or a collection of data. The retriever is designed to work with a vector database that stores data in the form of embeddings.

Embeddings are numerical representations of data (in this case, text data split into smaller chunks) that capture the semantic meaning of the original content. These embeddings are created using models like the ones provided by OpenAI (for example, through `OpenAIEmbeddings`), which convert textual content into high-dimensional vectors that numerically represent the semantic content of the text.

The vector database, in this context represented by `Chroma`, is a specialized database that stores these embeddings. The primary function of this database is to enable efficient similarity searches. That is, given a query in the form of an embedding, the database can quickly find and return the most similar embeddings (and by extension, the corresponding chunks of text or documents) stored within it.

When the code sets the `retriever` as `vectorstore.as_retriever()`, it essentially initializes a retriever object with the capability to perform these similarity-based searches within the `Chroma` vector database. This means that the retriever can be used to find the most relevant pieces of information (text chunks, in this scenario) based on a query. This is particularly useful in applications that needs to retrieve information based on semantic similarity rather than exact keyword matches.

In [8]:
# Set the retriever
retriever = vectorstore.as_retriever()

#### Set the prompt
* In this case, we will get a pre-defined prompt from the LangSmith Prompt Hub.

In [9]:
# Prompt
prompt = hub.pull("rlm/rag-prompt")

In [10]:
# Let's print the prompt we have selected from the LangSmith Prompt Hub
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

* Option 2:

In [11]:
# from langchain.prompts import ChatPromptTemplate

# template = """Answer the question based only on the following context:
# {context}

# Question: {question}
# """

# prompt = ChatPromptTemplate.from_template(template)

#### Determine which Foundation LLM we will use

In [12]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

#### Define a pre-processing to better format the document
* This step is optional

In [13]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

#### Define the RAG chain
Pay attention to this details:
* See how we use the retriever as context
* See how we use the pre-processing function
* See how we use the StrOutputParser to format the answer as a string

In [14]:
# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Phase 3: Generation

#### We can start asking questions to our RAG application

In [15]:
# Question
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a technique used to break down complex tasks into smaller and simpler steps. This approach helps agents to plan and execute tasks more efficiently by dividing them into manageable components. Task decomposition can be achieved through various methods, such as using prompting techniques, task-specific instructions, or human inputs.'

# Advanced techniques to improve RAG Performance

# 1. Query translation 
* Improving a question before it might lead to a low quality anwser from our RAG application

#### Alternative ways to transform the initial question:
* Convert the question in multiple similar questions
    * Multi-Query
    * RAG Fusion
* Convert the question in (progressive or not) subquestions
* Step-back questions (step-back prompting): go one step back and make the questions that precede or originate the user's question. Example:
    * User question: where were Mozart born?
    * Step-back question: who was Mozart? 

## Query translation technique #1: Multi-Query
* Convert the question into several similar questions
* Get the RAG answer for each of them
* Analyze all the answers and produce a final one 

#### Create a prompt template to create 5 similar questions to the question provided by the user

In [17]:
# Multi Query: Different Perspectives
from langchain.prompts import ChatPromptTemplate
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""

prompt_perspectives = ChatPromptTemplate.from_template(template)

#### Define a chain to generate the 5 similar questions

In [18]:
generate_queries = (
    prompt_perspectives 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

#### Get the RAG answer for each of them

In [19]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv032924/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


6

#### Analyze all the answers and produce a final one

In [20]:
from operator import itemgetter

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

"Task decomposition for LLM agents involves breaking down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks. This process allows the agent to transform big tasks into multiple manageable tasks and shed light on the interpretation of the model's thinking process."

## Query translation technique #2: RAG Fusion
* Same as Multi-Query, but ranking the resulting questions.

#### Same as before: create a prompt template to create 4 similar questions to the question provided by the user

In [21]:
# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

#### Same as before: define a chain to generate the 4 similar questions

In [22]:
generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

The following code demonstrates an implementation of the Reciprocal Rank Fusion (RRF) algorithm, leveraging functions from the `langchain.load` module for serialization and deserialization of documents. The RRF algorithm is used for combining the rankings of documents from multiple sources (or search results) into a single, consolidated ranking. The purpose is to enhance the quality of the search results by leveraging the strengths of multiple ranking systems. 

Here's a breakdown of the code:

#### Function `reciprocal_rank_fusion`
- **Parameters**:
  - `results`: A list of lists, where each inner list contains documents ranked according to their relevance from different sources or queries.
  - `k`: An optional parameter used in the RRF formula to control the influence of rank on the score. The default value is 60, which is a common choice in RRF implementations.

- **Process**:
  1. **Initialization**: A dictionary named `fused_scores` is created to store the cumulative score of each unique document across all rankings.
  2. **Ranking and Scoring**:
     - The code iterates over each list of ranked documents (`results`).
     - For each document in these lists, it calculates a score based on its rank using the RRF formula: `1 / (rank + k)`, where `rank` is the document's position in the list (starting from 0).
     - The document is serialized into a string (`doc_str`) using `dumps`, and this string serves as a key in the `fused_scores` dictionary. This approach assumes that documents can be uniquely identified after serialization.
     - If the document is new to `fused_scores`, it's initialized with a score of 0. Then, the RRF score is added to the document's existing score in the dictionary.
  3. **Sorting**: After processing all lists, the documents are sorted in descending order based on their cumulative scores in `fused_scores`.

- **Output**: The function returns the reranked results as a list of tuples, with each tuple containing the deserialized document and its fused score, sorted from highest to lowest score.

### RRF Algorithm in Simple Terms
The RRF algorithm works under the principle that documents ranked highly across multiple searches are likely more relevant than those ranked highly in just one. By inversely weighting the ranks (giving higher value to lower ranks), it ensures that documents consistently ranked well across different lists are prioritized in the final consolidated list.

### Usage Example
- **Chain of Operations**: The code snippet demonstrates how `reciprocal_rank_fusion` could be integrated into a larger pipeline (`retrieval_chain_rag_fusion`) that generates queries, retrieves documents based on those queries, and then applies the RRF algorithm to the results.
- **Invocation**: The `invoke` method is used to execute this pipeline with a specific `question`, and `len(docs)` gives the total number of documents after reranking.

This example showcases a practical application of RRF in enhancing search or retrieval systems by aggregating and refining results from multiple sources.

In [23]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

5

## Query translation technique #3: Decomposition
* Decompose de question into progressive sub-questions.

#### Create a prompt template to create 3 subquestions from the question provided by the user

In [24]:
# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [25]:
# LLM
llm = ChatOpenAI(temperature=0)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "What are the main components of an LLM-powered autonomous agent system?"

# These are the subquestions:
questions = generate_queries_decomposition.invoke({"question":question})

In [26]:
#print the subquestions:
questions

['1. What is LLM technology and how does it work in autonomous agent systems?',
 '2. What are the specific components that make up an autonomous agent system powered by LLM?',
 '3. How do the main components of an LLM-powered autonomous agent system interact with each other to achieve autonomy and intelligence?']

#### Option 1: answer each of the subquestions using the response to the previous subquestion as context.

In [27]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [28]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [29]:
answer

"The main components of an LLM-powered autonomous agent system interact with each other in a coordinated manner to achieve autonomy and intelligence. \n\n1. Planning: The agent utilizes planning to break down complex tasks into smaller subgoals, enabling efficient handling of tasks. This component interacts with subgoal decomposition to ensure that tasks are divided into manageable steps for better performance on complex tasks.\n\n2. Subgoal Decomposition: Tasks are decomposed into smaller and simpler steps, which enhances the model's performance on complex tasks. This component works in conjunction with planning to ensure that the agent can efficiently handle tasks by breaking them down into manageable subgoals.\n\n3. Reflection and Refinement: The agent engages in self-criticism and self-reflection over past actions, learning from mistakes and refining its approach for future tasks. This component interacts with planning and subgoal decomposition to continuously improve the agent's p

#### Option 2: answer each of the subquestions individually (not using the response to the previous subquestion as context).
* Useful when the subquestions are not dependent from each other.

In [30]:
# Answer each sub-question individually 

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [31]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

'The main components of an LLM-powered autonomous agent system include the Large Language Model (LLM) as the core controller, planning for task decomposition, reflection for self-criticism and learning from mistakes, and refinement for improving future actions. Additionally, the reliability of the natural language interface is crucial for communication between the LLM and external components. These components work together to enable autonomous functionality, allowing the agent to break down complex tasks, learn from errors, and continuously improve its performance.'

## Query translation technique #4: Step-back questions (also called step-back prompting)
* Step-back questions (step-back prompting): go one step back and make the questions that precede or originate the user's question. Example:
    * User question: where were Mozart born?
    * Step-back question: who was Mozart? 

In [32]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)


In [33]:
generate_queries_step_back = prompt | ChatOpenAI(temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'What is the process of breaking down tasks for LLM agents?'

In [34]:
# Response prompt 
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition for LLM agents refers to the process of breaking down complex tasks into smaller, more manageable subgoals. This approach enables LLM-powered autonomous agents to efficiently handle intricate tasks by dividing them into simpler steps. By decomposing tasks, the agent can effectively plan and execute each subgoal, leading to the successful completion of the overall task.\n\nIn the context of LLM-powered autonomous agent systems, task decomposition plays a crucial role in enhancing the performance and problem-solving capabilities of the agent. By breaking down large tasks into smaller components, the agent can focus on addressing one subgoal at a time, leading to a more systematic and organized approach to task completion.\n\nOne common technique used for task decomposition in LLM agents is the Chain of Thought (CoT) method. CoT prompts the model to "think step by step," encouraging it to decompose complex tasks into smaller and simpler steps. This approach leverages t

## Query Translation Technique #5: The HyDE Technique
* Given the user's question, create a fake document that would answer it properly.
* Then go to the vector database and find documents similar to the fake document. 

In [35]:
# HyDE document generation
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_docs_for_retrieval = (
    prompt_hyde | ChatOpenAI(temperature=0) | StrOutputParser() 
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})

'Task decomposition is a fundamental concept in the field of reinforcement learning and artificial intelligence, particularly for Large Language Models (LLMs) agents. Task decomposition refers to the process of breaking down a complex task into smaller, more manageable sub-tasks that can be solved independently or sequentially. \n\nIn the context of LLM agents, task decomposition plays a crucial role in improving the efficiency and effectiveness of the learning process. By decomposing a complex task into smaller sub-tasks, LLM agents can focus on solving each sub-task individually, which can lead to faster learning and better performance overall.\n\nFurthermore, task decomposition allows LLM agents to leverage their language understanding capabilities to interpret and generate instructions for each sub-task. This can help the agent to better understand the overall task structure and requirements, leading to more accurate and effective decision-making.\n\nOverall, task decomposition for

In [36]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever 
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be

In [37]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs,"question":question})

'Task decomposition for LLM agents involves breaking down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks. This process allows the agent to utilize more test-time computation to decompose hard tasks into smaller and simpler steps, ultimately improving the quality of final results.'

# Routing
Purpose: To direct the question to the appropriate data source (such as a vector database, relational database, or graph database) or the appropriate prompt or any other routing option.

Types of Routing:
* Logical Routing: Uses the LLM's knowledge of data sources to decide the best destination for a query.
* Semantic Routing: Involves embedding the question and prompts, calculating similarity, and selecting the prompt with the highest similarity for routing. For example: based on the question, use the Math prompt or the Physics prompt.

#### Logical Routing

In [38]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call 
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv032924/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [39]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})

In [40]:
result

RouteQuery(datasource='python_docs')

In [41]:
result.datasource

'python_docs'

In [42]:
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for js_docs"
    else:
        ### Logic here 
        return "golang_docs"

from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)

In [43]:
full_chain.invoke({"question": question})

'chain for python_docs'

#### Semantic Routing

In [44]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

# Embed prompts
embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt 
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

print(chain.invoke("What's a black hole"))

Using PHYSICS
A black hole is a region in space where the gravitational pull is so strong that nothing, not even light, can escape from it. It is formed when a massive star collapses in on itself, creating a singularity with infinite density at its center. Black holes have a boundary called the event horizon, beyond which nothing can escape. They are mysterious and fascinating objects in the universe that have many interesting properties and effects on their surroundings.


# Query structuring

Transform the query from natural language to some query syntax.

In the example below, we transform the query from natural language to the query syntax according to the schema we have defined.

In [47]:
from langchain_community.document_loaders import YoutubeLoader

docs = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=pbAd8O1Lvm4", add_video_info=True
).load()

docs[0].metadata

{'source': 'pbAd8O1Lvm4',
 'title': 'Self-reflective RAG with LangGraph: Self-RAG and CRAG',
 'description': 'Unknown',
 'view_count': 13798,
 'thumbnail_url': 'https://i.ytimg.com/vi/pbAd8O1Lvm4/hq720.jpg',
 'publish_date': '2024-02-07 00:00:00',
 'length': 1058,
 'author': 'LangChain'}

In [48]:
import datetime
from typing import Literal, Optional, Tuple
from langchain_core.pydantic_v1 import BaseModel, Field


class TutorialSearch(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    content_search: str = Field(
        ...,
        description="Similarity search query applied to video transcripts.",
    )
    title_search: str = Field(
        ...,
        description=(
            "Alternate version of the content search query to apply to video titles. "
            "Should be succinct and only include key words that could be in a video "
            "title."
        ),
    )
    min_view_count: Optional[int] = Field(
        None,
        description="Minimum view count filter, inclusive. Only use if explicitly specified.",
    )
    max_view_count: Optional[int] = Field(
        None,
        description="Maximum view count filter, exclusive. Only use if explicitly specified.",
    )
    earliest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Earliest publish date filter, inclusive. Only use if explicitly specified.",
    )
    latest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Latest publish date filter, exclusive. Only use if explicitly specified.",
    )
    min_length_sec: Optional[int] = Field(
        None,
        description="Minimum video length in seconds, inclusive. Only use if explicitly specified.",
    )
    max_length_sec: Optional[int] = Field(
        None,
        description="Maximum video length in seconds, exclusive. Only use if explicitly specified.",
    )

    def pretty_print(self) -> None:
        for field in self.__fields__:
            if getattr(self, field) is not None and getattr(self, field) != getattr(
                self.__fields__[field], "default", None
            ):
                print(f"{field}: {getattr(self, field)}")

In [49]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
Given a question, return a database query optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = llm.with_structured_output(TutorialSearch)
query_analyzer = prompt | structured_llm

In [51]:
#type(out)

In [52]:
query_analyzer.invoke({"question": "rag from scratch"}).pretty_print()

content_search: rag from scratch
title_search: rag from scratch


In [53]:
query_analyzer.invoke(
    {"question": "videos on chat langchain published in 2023"}
).pretty_print()

content_search: chat langchain
title_search: chat langchain
earliest_publish_date: 2023-01-01
latest_publish_date: 2024-01-01


In [54]:
query_analyzer.invoke(
    {
        "question": "how to use multi-modal models in an agent, only videos under 5 minutes"
    }
).pretty_print()

content_search: multi-modal models agent
title_search: multi-modal models agent
max_length_sec: 300
